In [2]:
import requests
import re

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

from time import sleep

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [20]:
lista_actores = ['Danny Trejo','Clifton Powell', 'Leonardo Di Caprio', 'Byom']# 'Adam Sandler','Emily Rose Hernandez','Oscar Nuñez'] #Adam Sandler','Danny Trejo','Clifton Powell', 'Emily Rose Hernandez', 'Craig Edwards']
lista_detalles_actores = []
datos_actor = {}
url_actores=(f"https://www.imdb.com/")
driver = webdriver.Chrome()
driver.get(url_actores)
driver.maximize_window()
sleep(2)
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
sleep(3)
for actor in lista_actores:
    driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)
    sleep(5)
    datos_actor["nombre_actor"] = actor
    driver.execute_script("window.scrollTo(0,400)")
    sleep (3)
    driver.find_element('xpath','//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div/a').click()
    sleep (3)
    try: 
        resultado_que_hace = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
        lista_que_hace = resultado_que_hace.split("\n")
        datos_actor["que_hace"] = lista_que_hace
    except:
        print("Error al encontrar la seccion 'que hace'")
        datos_actor["que_hace"] = "None"
    sleep(2)
    try:
        driver.execute_script("window.scrollTo(0,500)")
        anno_nacimiento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text
        resultado_anno_nacimiento = re.search(r'\b\d{4}\b', anno_nacimiento).group()
        datos_actor["anno_nacimiento"] = resultado_anno_nacimiento
        #anno_nacimiento antiguo:
        #datos_actor["anno_nacimiento"] = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text.split(" ")[-1]
    except:
        print("Error al extrear el año de nacimiento")  #NUEVO: //*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li
        datos_actor['anno_nacimiento'] = "0"            #PREMIO ANTIGUOS: //*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li
    sleep(5)                                            #//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul/li/span
    #try:
    i = 500
    while i<2000:
        try:
            driver.execute_script(f"window.scrollTo(0,{i})")
            sleep(3)                                              #//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul
            
            try:
                texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul').text #.split(" ")[0]
                break
            except:
                texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul').text #.split(" ")[0]
                break
        except:
            i+=500
    try:
        #texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li').text
        #.split(" ")[0]
            #try:
        cantidad_premios = re.search(r'(\d+)\s+premios?',texto_premios, re.IGNORECASE).group(1)
        datos_actor['premios'] = cantidad_premios
    except:
        print("Error al extrear los premios")
        datos_actor['premios'] = "0"
        
    
    driver.execute_script(f"window.scrollTo({i},0)")
        #resultado_premios = re.search(r'(\d+)\s+premios?',texto_premios, re.IGNORECASE).group(1)

        #re.search(r'(\d+)\s*premio[s]?',texto_premios, re.IGNORECASE).group(1)
        # PREMIO FUNCIONANTE CON PREMIOS: resultado_premios = re.search(r'(\d+)\s+premios',texto_premios, re.IGNORECASE).group(1)
    # except:
    #     print("Error al extrear los premios")
    #     datos_actor['premios'] = "0"
    sleep(4)
    listas_pelis_conocido_por = []
    for x in range(1,5):
        try:
            driver.execute_script("window.scrollTo(0,2000)")
            sleep(3)                                                     #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-af50cbe8-0.fqoRPX > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top
            resultado_conocido_por = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-af50cbe8-0.fqoRPX > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({x}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top').text
            listas_pelis_conocido_por.append(resultado_conocido_por)
            #resultado_conocido_por = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a').text
            #listas_pelis_conocido_por.append(resultado_conocido_por.split("\n")[0])            
        except:
            print("Error al encontrar las peliculas")
            listas_pelis_conocido_por.append("None")
    sleep(2)
    datos_actor["peliculas"] = listas_pelis_conocido_por    
    # try:
    #     tupla_actor = (datos_actor['nombre_actor'], int(datos_actor['anno_nacimiento']), datos_actor["peliculas"], datos_actor["que_hace"], int(datos_actor['premios'])) 
    # except:
    #     print("Error al crear la tupla. Creamos tupla con null")
    #     tupla_actor = (datos_actor['nombre_actor'], 0, ["None"], ["None"], 0)
    
    lista_detalles_actores.append(datos_actor)
    datos_actor = {}
        
print("----------------")
    
print(lista_detalles_actores)

Error al extrear el año de nacimiento
----------------
[{'nombre_actor': 'Danny Trejo', 'que_hace': ['Reparto', 'Producción', 'Equipos adicionales'], 'anno_nacimiento': '1944', 'premios': '18', 'peliculas': ['Machete', 'Abierto hasta el amanecer', 'Predators', 'Machete Kills']}, {'nombre_actor': 'Clifton Powell', 'que_hace': ['Reparto', 'Producción', 'Dirección'], 'anno_nacimiento': '1956', 'premios': '2', 'peliculas': ['Ray', 'Deep Rising: El misterio de las profundidades', 'Next Friday', 'Hora punta']}, {'nombre_actor': 'Leonardo Di Caprio', 'que_hace': ['Producción', 'Reparto', 'Guion'], 'anno_nacimiento': '1974', 'premios': '106', 'peliculas': ['Origen', 'El lobo de Wall Street', 'Infiltrados', 'Titanic']}, {'nombre_actor': 'Byom', 'que_hace': ['Reparto', 'Producción', 'Dirección de la segunda unidad o asistencia a dirección'], 'anno_nacimiento': '0', 'premios': '106', 'peliculas': ["Mom's Coming", 'Lage Na Mon', 'Love Driver', 'Jai Ho Madhya Pradesh']}]


In [21]:
import pandas as pd

In [22]:
df = pd.DataFrame(lista_detalles_actores)
df

,nombre_actor,que_hace,anno_nacimiento,premios,peliculas
0,Danny Trejo,"[Reparto, Producción, Equipos adicionales]",1944,18,"[Machete, Abierto hasta el amanecer, Predators..."
1,Clifton Powell,"[Reparto, Producción, Dirección]",1956,2,"[Ray, Deep Rising: El misterio de las profundi..."
2,Leonardo Di Caprio,"[Producción, Reparto, Guion]",1974,106,"[Origen, El lobo de Wall Street, Infiltrados, ..."
3,Byom,"[Reparto, Producción, Dirección de la segunda ...",0,106,"[Mom's Coming, Lage Na Mon, Love Driver, Jai H..."


In [3]:
lista_id_peliculas = ['tt10225380','tt10844184','tt10928904', 'tt11078340']
lista_actores = []
url_actores=("https://www.imdb.com/")
driver = webdriver.Chrome()
driver.get(url_actores)
driver.maximize_window()
sleep(2)
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
sleep(2)
for pelicula in lista_id_peliculas:
    print(pelicula)
    try:
        driver.find_element("css selector", "#suggestion-search").send_keys(pelicula, Keys.ENTER)
    except:
        print("Pelicula no encontrada")
    sleep(2)
    #driver.execute_script("window.scrollTo(0,1000)")
    for i in range(1,11):
        try:       
            actor = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})').text.split("\n")[0]
            lista_actores.append(actor)   
        except:
            print(f"Error al encontrar los 10 primeros actores de la pelicula {pelicula}") 
    sleep(2)
    driver.back()
    sleep(2)
print(lista_actores)        

tt10225380
tt10844184
Error al encontrar los 10 primeros actores de la pelicula tt10844184
tt10928904
tt11078340
['Juanjo Martínez', 'Solange Chilinski', 'Sol Morandini', 'Carlos Cadel', 'Laura Quiroga', 'Luz Fernandez', 'Natalia Goncalves', 'Leandro Bombicino', 'Facundo Serrano', 'Denise Depauli', 'Danny Trejo', 'Clifton Powell', 'Craig Edwards', 'Emily Rose Hernandez', 'Matthew Clark "Ghost"', 'Tesha Clark', 'Paul Edward Davies', 'Rich Brodie', 'Karina Encarnacion', 'Dylan Playfair', 'Clifton Powell', 'Hector David Jr.', 'Bronsonn Taylor', 'Emily Rose Hernandez', 'Raven Armando Astone', 'David Siracusa', 'Alexander Arzu', 'Nate Bynum', 'John Ozuna', 'Redd Nicholson', 'Luna Wolf', 'Carly Halse', 'Paris Rivers', 'Lewis Cooper', 'Cy Henty', 'Richard Fysh', 'Grant Kempster', 'Pauline Peart', 'Matthew Baunsgard']
